# Rb87 Polarizability Calcs

In [8]:
#### libraries
from numpy import *
import matplotlib.pyplot as plt
from arc import *
from sympy.physics.wigner import wigner_6j,wigner_3j,clebsch_gordan

#### local files
from physconsts import *
from rbconsts import *
# from amophys import cgsToSI

In [5]:
# a dictionary of hf levels through 5p3/2. "nu" in [GHz], w.r.t. 5s1/2 COM
hf_levels = [
    {'n': 5, 'L': 0, 'J': 0.5, 'F': 1, 'nu': -4.271676631815196},
    {'n': 5, 'L': 0, 'J': 0.5, 'F': 2, 'nu': 2.563005979089114},
    {'n': 5, 'L': 1, 'J': 0.5, 'F': 1, 'nu': 377106.953053064},
    {'n': 5, 'L': 1, 'J': 0.5, 'F': 2, 'nu': 377107.769709364},
    {'n': 5, 'L': 1, 'J': 1.5, 'F': 0, 'nu': 384230.1823946245},
    {'n': 5, 'L': 1, 'J': 1.5, 'F': 1, 'nu': 384230.2546166565},
    {'n': 5, 'L': 1, 'J': 1.5, 'F': 2, 'nu': 384230.4115571805},
    {'n': 5, 'L': 1, 'J': 1.5, 'F': 3, 'nu': 384230.6782093585}
]

In [6]:
## example syntax for arc matrix element calcs
# #Radial Matrix element R_{nlj\rightarrown'l'j'}
# atom.getRadialMatrixElement(n1,l1,j1,n2,l2,j2)
# #Reduced Matrix Element <l||er||l'>
# atom.getReducedMatrixElementL(n1,l1,j1,n2,l2,j2)
# #Reduced Matrix Element <j||er||j'>
# atom.getReducedMatrixElementJ(n1,l1,j1,n2,l2,j2)
# #Angular Coupling <nljmj|er|n'l'j'mj'>
# atom.getDipoleMatrixElement(n1,l1,j1,mj1,n2,l2,j2,mj2,q)

In [7]:
rb = Rubidium(87)
# D2 line dipole matrix element, HF basis
n1,l1,j1,f1,mf1 = [5,0,1/2,2,0] 
n2,l2,j2,f2,mf2 = [5,1,3/2,3,0]
rb.getReducedMatrixElementJ(n1,l1,j1,n2,l2,j2)

5.956

In [8]:
# 5p1/2
(omega_D1/(2*pi) + 306246110)/1e9 # 

377107.769709364

In [9]:
# 5p3/2
(omega_D2/(2*pi) - 302073888)/1e9 # F
(omega_D2/(2*pi) - 229851856)/1e9 # F1
(omega_D2/(2*pi) - 72911332)/1e9 # F2
(omega_D2/(2*pi) + 193740846)/1e9 # F3

384230.6782093585

In [10]:
lambda_list = array([2*pi*c/w for w in w_list])/(1e-6)
plt.plot(lambda_list,alpha0*(ee*a0)**2/EH)
plt.ylabel("arb")
plt.xlabel("$\lambda$ [$\mu$m]")
plt.show()

NameError: name 'w_list' is not defined

In [11]:
(ee*a0)**2/EH

1.6487772743567905e-41

In [12]:
JToeV(EH)/2

13.605693122994017

In [11]:
f_960 = 312.6574*10**12
l_960 = c/f_960
l_480 = l_960/2
f_780A = 384.2254*10**12
l_780A = c/f_780A
l_780A, l_480

(7.802515346460697e-07, 4.794264552830031e-07)

## Finite sum for $\alpha_0(\omega)$, 5s1/2 - J-M basis

In [31]:
# our atom
rb = Rubidium(87)

# the 5s1/2 state
level_a = hf_levels[1] 
n_a = level_a["n"]
j_a = level_a["J"] 
l_a = level_a["L"]
# w_a = 2*pi*level_a["E"]*1e9 # [rad/s], wrt zeropoint at 5s1/2 COM

l_b = l_a + 1 # for dipole-allowed transitions with l_a = 0
j_b_list = [ l_b - .5, l_b + .5] 
n_min = 5
n_max = 11 # exclusive max

# pts = 20

l_arr = [4.794264552830031e-07] # 1.064e-6] # selected wavelengths
w_arr = array([2*pi*c/l for l in l_arr]) # ang. freqs
mF_a = 0
q = 0
mF_b = mF_a + q

pts = len(l_arr)
alpha0 = zeros(pts) 
terms = 0
for i in range(pts):
    for n_b in range(n_min, n_max):
        for l_b in range(n_b): # runs through n_b - 1, inclusive
            for j_b in j_b_list: 
                
                # triangle and dipole-allowed conditions
                if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1: 
                    w_ba = 2*pi*(eVToGHz(rb.getEnergy(n_b, l_b, j_b) \
                                        - rb.getEnergy(n_a, l_a, j_a))*1e9)
                    matelem = abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b)*ee*a0)
#                     print(f"< n={n_a}, l={l_a}, j={j_a} | x | n'={n_b}, l'={l_b}, j'={j_b} >")
                    alpha0[i] += (2/(3*hbar*(2*j_a+1)))* \
                                    w_ba*matelem**2/(w_ba**2 - w_arr[i]**2) # Mark's Notes 9.11a
                    terms += 1
                    print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-6)}, {terms} terms in sum")
# until now, alpha0 is in SI units
# alpha0 # [Angst.**3] 1/(4*pi*e0*1e-6) # [cgs] to a.u.: EH/(ee*a0)**2

alpha0 ~= -8.776424467960309e-24, 1 terms in sum
alpha0 ~= -2.6977341449407057e-23, 2 terms in sum
alpha0 ~= -2.675345855205357e-23, 3 terms in sum
alpha0 ~= -2.6177271476240064e-23, 4 terms in sum
alpha0 ~= -2.6165524368212375e-23, 5 terms in sum
alpha0 ~= -2.61294421439118e-23, 6 terms in sum
alpha0 ~= -2.612696804064086e-23, 7 terms in sum
alpha0 ~= -2.61182270277221e-23, 8 terms in sum
alpha0 ~= -2.6115835576633552e-23, 9 terms in sum
alpha0 ~= -2.610857751460293e-23, 10 terms in sum
alpha0 ~= -2.6107363818084868e-23, 11 terms in sum
alpha0 ~= -2.6103586448076076e-23, 12 terms in sum


In [14]:
mine = alpha0[0]/(4*pi*e0*1e-6) # cgs
marks = 97e-24 # for 1064nm
print(mine,marks)
print(mine/marks-1)

9.992826185051465e-23 9.7e-23
0.03018826650015094


### AC Stark Shift on $|5 s_{1/2} F=2, m_F=0>$
due to the Rydberg 960 and 780A beams

In [25]:
alpha0_780A = 4*pi*e0*1e-6*1.1653070912238685e-18
alpha0_480 = 4*pi*e0*1e-6*4.794264552830031e-07

efield = lambda intensity: sqrt(2*intensity/(c*e0)) 

def beam_int(z, wx, wy, lmbda, P, xy_dz=0, r=0):
    zRx = pi*wx**2/lmbda
    zRy = pi*wy**2/lmbda
    
    wwx = wx**2*(1+(z/zRx)**2)
    wwy = wy**2*(1+(z/zRy)**2)
    return (P/(pi*wx*wy))*exp(-r**2/wwx)*exp(-r**2/wwy)/sqrt(wwx*wwy)
e780A = efield(beam_int(0, 6e-6, 8e-6, l_780A, 8e-6, xy_dz=200e-6))
e480 = efield(beam_int(0, 5e-6, 5e-6, l_480, 20e-3))

# acstark
acstark0 = -.25*(alpha0_780A*e780A**2 + alpha0_480*e480**2)
acstark0

-0.10234844913314059

## Finite sum for $\alpha_0(\omega)$, 5s1/2 - Hyperfine basis
This doesn't come out correctly.. :/

In [17]:
#TODO: refactor this to make it much more user friendly
# - define a function withs args for state (n, l, j, f), 
# the uppermost state (n', l', j', f'), and wavelengths
# and polarization of the incident light

# our atom
rb = Rubidium(87)

# the 5s1/2,F=2,mF=0 state 
level_a = hf_levels[1] 
n_a = level_a["n"]
j_a = level_a["J"] 
l_a = level_a["L"]
f_a = level_a["F"]
mf_a = 0 # I'm not sure these ever get used.. might be defined below
mf_b = 0



# for the L-S coupling part (states beyond 5p3/2)
l_b = l_a + 1 # for dipole-allowed transitions with l_a = 0
j_b_list = [ l_b - .5, l_b + .5] 

n_min = 5
n_mid = 6
n_max = 11 # exclusive max

l_arr = [1.064e-6] # selected wavelengths, [m]
w_arr = array([2*pi*c/l for l in l_arr]) # ang. freqs
mF_a = 0
q = 0
mF_b = mF_a + q


pts = len(l_arr)
alpha0 = zeros(pts) 
terms = 0
for i in range(len(l_arr)):

    for level_b in hf_levels: # loop over states up through 5p3/2 band (inclusive)
        
        n_b = level_b["n"]
        j_b = level_b["J"] 
        l_b = level_b["L"]
        f_b = level_b["F"]
        
        # triangle and dipole-allowed conditions
        if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1 and abs(f_b - f_a) == q: 
            
            w_ba = 2*pi*(level_b["nu"] - level_a["nu"])*1e9
                        
            # matrix elems squared for hyperfine terms (Mark's notes, after 9.17)
            h0 = (2*f_a + 1)**2*sum([abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b))**2 \
                    *wigner_6j(j_b,f_a,I,f_a,j_a,1)**2 for j_b in j_b_list])*(ee*a0)**2
            
#            print(f"<n={n_a},l={l_a},j={j_a},f={f_a}|x|n'={n_b},l'={l_b},j'={j_b},f'={f_b}>")
            alpha0[i] += (2/(3*hbar*(2*f_a+1)))*w_ba*h0 \
                            /(w_ba**2 - w_arr[i]**2) # Mark's Notes 9.16
            terms += 1
            print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-6)}, {terms} terms in sum")
            
    for n_b in range(n_mid, n_max):
        for l_b in range(n_b): # runs through n_b - 1, inclusive
            for j_b in j_b_list: 

                # triangle and dipole-allowed conditions
                if abs(j_b - j_a) <= 1 and abs(l_b - l_a) == 1: 
                    w_ba = 2*pi*(eVToGHz(rb.getEnergy(n_b, l_b, j_b) \
                                        - rb.getEnergy(n_a, l_a, j_a))*1e9)
                    matelem = abs(rb.getReducedMatrixElementJ(n_a, l_a, j_a, n_b, l_b, j_b)*ee*a0)
#                     print(f"< n={n_a}, l={l_a}, j={j_a} | x | n'={n_b}, l'={l_b}, j'={j_b} >")
                    alpha0[i] += (2/(3*hbar*(2*j_a+1)))* \
                                    w_ba*matelem**2/(w_ba**2 - w_arr[i]**2) # for q = 0. 
                    
                    terms += 1
#                     print(f"alpha0 ~= {alpha0[i]/(4*pi*e0*1e-6)}, {terms} terms in sum")
print(f"for {level_a}")

alpha0 ~= 3.468244812542488e-23, 1 terms in sum
alpha0 ~= 6.721196582390219e-23, 2 terms in sum
for {'n': 5, 'L': 0, 'J': 0.5, 'F': 2, 'nu': 2.563005979089114}


In [70]:
# 5s1/2,F=2: alpha0 ~= 6.746911307357744e-23, 12 terms in sum (with F=F')
# 5s1/2,F=1: alpha0 ~= 6.736462307586051e-23, 12 terms in sum (with F=F')

In [52]:
3.682966809168338e-23, 16 terms in sum

0.5

In [37]:
(hf_levels[4]["nu"] - hf_levels[0]["nu"])*1e9

384234454071256.3

In [40]:
help(hf_coupling)

<function amophys.hf_coupling(F, mF, J, q, FF, mFF, JJ, I, RME=None)>

In [57]:
h0 = (2*f_a + 1)**2*sum([
        rb.getReducedMatrixElementJ(
            n_a, l_a, j_a, n_b, l_b, j_b
            )*wigner_6j(j_b,f_a,I,f_a,j_a,1)*ee*a0
        for j_b in j_b_list])**2

6j-Symbol is not triangular!
6j-Symbol is not triangular!


In [58]:
del h0